In [2]:
import pandas as pd
import numpy as np
from numba import njit

In [52]:
sept = pd.read_pickle("sept.pickle")
sept

,text,str,verse,chapter,book,rmac
0,εν,G1722,1,1,1,prep
1,αρχη,G746,1,1,1,n-dsf
2,εποιησεν,G4160,1,1,1,v-aai-3s
3,ο,G3588,1,1,1,t-nsm
4,θεος,G2316,1,1,1,n-nsm
...,...,...,...,...,...,...
475395,και,G2532,6,4,39,conj
475396,παταξω,G3960,6,4,39,v-fai-1s
475397,την,G3588,6,4,39,t-asf
475398,γην,G1065,6,4,39,n-asf


In [4]:
nt = pd.read_pickle("tisch")  # from Isaac's PS1
nt

,rmac,str,gr,STYLE,Speech:,Case:,Number:,Gender:,Degree:,Form:,Tense:,Voice:,Mood:,Person:
0,a-apf,2117.0,εὐθείας,None,Adjective,Accusative,Plural,Feminine,NaN,NaN,NaN,NaN,NaN,NaN
1,a-apf,3956.0,πάσας,None,Adjective,Accusative,Plural,Feminine,NaN,NaN,NaN,NaN,NaN,NaN
2,a-apf,4183.0,πολλὰς,None,Adjective,Accusative,Plural,Feminine,NaN,NaN,NaN,NaN,NaN,NaN
3,a-apf,3956.0,πάσας,None,Adjective,Accusative,Plural,Feminine,NaN,NaN,NaN,NaN,NaN,NaN
4,a-apf,5140.0,τρεῖς,None,Adjective,Accusative,Plural,Feminine,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137486,x-nsn,5100.0,τι,None,indefinite,Nominative,Singular,Neuter,NaN,NaN,NaN,NaN,NaN,NaN
137487,x-nsn,5100.0,"τι,",None,indefinite,Nominative,Singular,Neuter,NaN,NaN,NaN,NaN,NaN,NaN
137488,x-nsn,5100.0,τι,None,indefinite,Nominative,Singular,Neuter,NaN,NaN,NaN,NaN,NaN,NaN
137489,x-nsn,5100.0,τι,None,indefinite,Nominative,Singular,Neuter,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
@njit(parallel=True)
def _find_matching_ngrams(left, right, min_n=5):
    """
    Helper function for `find_matching_ngrams`
    This does the actual computation, operating on numpy arrays
    """
    found_values = []
    lidx = 0
    while lidx < len(left):
        for ridx in range(len(right)):
            length = 0
            while right[ridx + length] == left[lidx + length]:
                length += 1
            if length >= min_n:
                found_values.append((lidx, ridx, length))
                lidx += length - 1
        lidx += 1
    found = np.array(found_values)
    return found[:, 0], found[:, 1], found[:, 2]


def find_matching_ngrams(left, right, min_n=5):
    """
    Itererate `n` from `max_n` to `min_n` (inclusive). For each n-gram in `left`, look for matching occurances in `right`
    """
    left_ids, right_ids, lengths = _find_matching_ngrams(
        np.array(left), np.array(right), min_n
    )
    return pd.DataFrame(
        {
            "left_id": left_ids,
            "left_len": lengths,
            "right_id": right_ids,
            "right_len": lengths,
        }
    )

In [21]:
find_matching_ngrams(sept["str"], nt["str"])

/home/finally/Old-Testament-References/env/lib/python3.13/site-packages/numba/core/typed_passes.py:338: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../tmp/ipykernel_5843/48240416.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


,left_id,left_len,right_id,right_len
0,4764,6,13938,6
1,4804,5,13937,5
2,5031,6,13938,6
3,5036,5,13939,5
4,12896,6,13938,6
...,...,...,...,...
102,254363,6,13938,6
103,257141,6,13938,6
104,343913,5,13938,5
105,363867,5,13938,5
